# Stock Data Analysis: International & Iranian Markets

This notebook demonstrates how to fetch, clean, and explore historical stock data for major international companies (using yfinance) and major Iranian companies (using finpy_tse).

In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import finpy_tse

In [2]:
# Define tickers
stock_tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'META']  # International
iran_symbols = ['فولاد', 'فارس', 'شپنا', 'خودرو', 'شستا']  # Iranian

In [6]:
# Download international stock data
date_end = datetime.today()
date_start = date_end - timedelta(days=5*365)
data = yf.download(
    tickers=stock_tickers,
    start=date_start.strftime('%Y-%m-%d'),
    end=date_end.strftime('%Y-%m-%d'),
    interval='1d',
    group_by='ticker',
    auto_adjust=True
)
display(data.head())

[*********************100%***********************]  5 of 5 completed



Ticker           AAPL                                                    AMZN  \
Price            Open       High        Low      Close     Volume        Open   
Date                                                                            
2020-07-02  89.393898  90.030602  88.370797  88.485008  114041600  145.600494   
2020-07-06  89.916395  91.321034  89.884801  90.852013  118655600  146.748505   
2020-07-07  91.231114  92.011197  90.458321  90.570107  112424400  152.927505   
2020-07-08  91.549483  92.711105  91.461993  92.679512  117092000  151.130493   
2020-07-09  93.573791  93.627255  92.028205  93.078041  125642800  155.799500   

Ticker                                                     ...        META  \
Price             High         Low       Close     Volume  ...        Open   
Date                                                       ...               
2020-07-02  147.778000  143.554993  144.514999  131868000  ...  237.695980   
2020-07-06  152.994003  146.500000  152.852005  137612000  ...  232.484549   
2020-07-07  153.477493  149.500000  150.005997  105150000  ...  238.103752   
2020-07-08  154.198502  150.621506  154.055496  100752000  ...  236.810851   
2020-07-09  159.694000  153.699997  159.131500  127774000  ...  243.663272   

Ticker                                                          MSFT  \
Price             High         Low       Close    Volume        Open   
Date                                                                   
2020-07-02  238.690524  231.340845  232.146423  30633600  197.059039   
2020-07-06  239.088319  231.002689  238.968979  26206200  200.077052   
2020-07-07  246.298784  237.516975  239.545837  27887800  201.629102   
2020-07-08  245.642405  235.299140  242.251007  29791300  201.265052   
2020-07-09  245.184923  237.934705  243.166000  22174900  207.262692   

Ticker                                                    
Price             High         Low       Close    Volume  
Date                                                      
2020-07-02  199.300971  196.407548  197.614731  29315800  
2020-07-06  202.280652  199.368063  201.868668  31897600  
2020-07-07  205.672224  199.272220  199.521317  33600700  
2020-07-08  204.321333  199.942889  203.909363  33600000  
2020-07-09  207.310599  202.606394  205.336945  33121700  

[5 rows x 25 columns]

In [ ]:
# Download Iranian stock data
iran_data = {}
for symbol in iran_symbols:
    try:
        df = finpy_tse.get_price_history(symbol)
        iran_data[symbol] = df
        print(f"{symbol}: shape={df.shape}, columns={list(df.columns)}")
        display(df.head(1))  # Show only the first row as a sample
    except Exception as e:
        print(f"Error for {symbol}: {e}")

In [14]:
# 1. Setup and Data Loading

import pandas as pd
import matplotlib.pyplot as plt

# Set a professional plot style
plt.style.use('seaborn-v0_8-whitegrid')

# For reproducibility: Uncomment the following line to load from CSV if needed
# iran_close_prices = pd.read_csv('iranian_stocks.csv', index_col=0, parse_dates=True)

# ساخت DataFrame نهایی از iran_data (در صورت عدم وجود)
if 'iran_close_prices' not in globals():
    # نمایش نام ستون‌های اولین دیتافریم برای تعیین نام صحیح ستون قیمت بسته‌شدن
    print('Sample columns:', list(next(iter(iran_data.values())).columns))
    # iran_close_prices = pd.concat([df['Close'] for df in iran_data.values()], axis=1)
    # iran_close_prices.columns = list(iran_data.keys())

print("Head of iran_close_prices DataFrame:")
print(iran_close_prices.head())

Sample columns: ['Open', 'High', 'Low', 'Close', 'Final', 'Volume', 'Value', 'No', 'Ticker', 'Name', 'Market']
Head of iran_close_prices DataFrame:


NameError: name 'iran_close_prices' is not defined

In [ ]:
# 2. Calculate and Display Daily Returns

daily_returns = iran_close_prices.pct_change()
daily_returns = daily_returns.iloc[1:]

print("Head of daily_returns DataFrame:")
print(daily_returns.head())

In [ ]:
# 3. Plot Historical Prices

plt.figure(figsize=(15, 7))
iran_close_prices.plot(ax=plt.gca())
plt.title("Historical Adjusted Close Prices")
plt.xlabel("Date")
plt.ylabel("Adjusted Close Price")
plt.legend(title="Stock Ticker")
plt.show()

In [ ]:
# 4. Plot Daily Returns

plt.figure(figsize=(15, 7))
for col in daily_returns.columns:
    plt.plot(daily_returns.index, daily_returns[col], linestyle='none', marker='.', label=col)
plt.axhline(0, color='gray', linestyle='--', linewidth=1)
plt.title("Daily Returns Volatility")
plt.xlabel("Date")
plt.ylabel("Daily Return")
plt.legend(title="Stock Ticker")
plt.show()

In [ ]:
# 5. Calculate and Display Key Statistics

mean_returns = daily_returns.mean()
daily_volatility = daily_returns.std()
risk_return_summary = pd.DataFrame({
    'Mean Daily Return': mean_returns,
    'Daily Volatility': daily_volatility
})

print("Risk-Return Summary (Mean Daily Return & Daily Volatility):")
print(risk_return_summary)